In [1]:
def bytes_info(size): #выводит размер файла в байтах, килобайтах и мегабайтах
    print('Размер файла:',  size, 'байт или ', round(size/1024, 3), 'Кб или ', round(size/1024**2, 3), 'Мб')

def compare_size(old, new): #выводит процент, который составляет новый размер файла от старого
    print('Новый размер файла составляет ', round(new/old, 3)*100, '% от старого')

In [2]:
# загружаем текст и вычисляем его исходный размер
with open('../text_for_decap.txt', 'r', encoding='ascii') as f:
        text = f.read()

main_size=len(bytearray(text, 'ascii'))
bytes_info(main_size)

text1=text
text2=text.lower()
with open('text_decape.txt', 'w', encoding='ascii') as f:
        f.write(text2)
text3=text1[::-1]
with open('text_for_decap_reverse.txt', 'w', encoding='ascii') as f:
        f.write(text3)
text4=text2[::-1]
with open('text_decap_reverse.txt', 'w', encoding='ascii') as f:
        f.write(text4)


Размер файла: 3171529 байт или  3097.196 Кб или  3.025 Мб


In [3]:
from collections import defaultdict

# === Специальные символы ===
EOF_SYMBOL = '<EOF>'
ESCAPE_SYMBOL = '<ESC>'

# === FrequencyTable для арифметического кодирования ===
class FrequencyTable:
    def __init__(self, symbols):
        self.symbols = list(symbols)
        self.frequencies = {s: 1 for s in self.symbols}
        self._build_cum_freqs()

    def set_freqs(self, freqs):
        self.frequencies = dict(freqs)
        self.symbols = list(freqs.keys())
        self._build_cum_freqs()

    def _build_cum_freqs(self):
        self.cum_freqs = {}
        total = 0
        for s in self.symbols:
            self.cum_freqs[s] = total
            total += self.frequencies[s]
        self.total = total

    def get_low(self, symbol):
        return self.cum_freqs[symbol]

    def get_high(self, symbol):
        return self.cum_freqs[symbol] + self.frequencies[symbol]

    def get_total(self):
        return self.total


# === Арифметический кодер ===
class ArithmeticEncoder:
    def __init__(self, precision=32):
        self.low = 0
        self.high = (1 << precision) - 1
        self.precision = precision
        self.output = []
        self.pending_bits = 0

    def encode_symbol(self, symbol, freq_table):
        total = freq_table.get_total()
        low_count = freq_table.get_low(symbol)
        high_count = freq_table.get_high(symbol)
        range_ = self.high - self.low + 1
        self.high = self.low + (range_ * high_count // total) - 1
        self.low = self.low + (range_ * low_count // total)

        while True:
            if self.high < (1 << (self.precision - 1)):
                self._output_bit(0)
            elif self.low >= (1 << (self.precision - 1)):
                self._output_bit(1)
                self.low -= 1 << (self.precision - 1)
                self.high -= 1 << (self.precision - 1)
            elif self.low >= (1 << (self.precision - 2)) and self.high < 3 << (self.precision - 2):
                self.pending_bits += 1
                self.low -= 1 << (self.precision - 2)
                self.high -= 1 << (self.precision - 2)
            else:
                break
            self.low <<= 1
            self.high = (self.high << 1) | 1

    def _output_bit(self, bit):
        self.output.append(bit)
        for _ in range(self.pending_bits):
            self.output.append(1 - bit)
        self.pending_bits = 0

    def finish(self):
        self.pending_bits += 1
        if self.low < (1 << (self.precision - 2)):
            self._output_bit(0)
        else:
            self._output_bit(1)
        return self.output


# === Арифметический декодер ===
class ArithmeticDecoder:
    def __init__(self, bits, precision=32):
        self.bits = bits
        self.precision = precision
        self.low = 0
        self.high = (1 << precision) - 1
        self.value = 0
        self.index = 0
        for _ in range(precision):
            self.value = (self.value << 1) | self._read_bit()

    def _read_bit(self):
        if self.index < len(self.bits):
            b = self.bits[self.index]
            self.index += 1
            return b
        return 0

    def decode_symbol(self, freq_table):
        total = freq_table.get_total()
        range_ = self.high - self.low + 1
        offset = self.value - self.low
        value = ((offset + 1) * total - 1) // range_

        for symbol in freq_table.symbols:
            if freq_table.get_low(symbol) <= value < freq_table.get_high(symbol):
                break
        else:
            raise ValueError("Symbol not found in frequency table")

        low_count = freq_table.get_low(symbol)
        high_count = freq_table.get_high(symbol)
        self.high = self.low + (range_ * high_count // total) - 1
        self.low = self.low + (range_ * low_count // total)

        while True:
            if self.high < (1 << (self.precision - 1)):
                pass
            elif self.low >= (1 << (self.precision - 1)):
                self.low -= 1 << (self.precision - 1)
                self.high -= 1 << (self.precision - 1)
                self.value -= 1 << (self.precision - 1)
            elif self.low >= (1 << (self.precision - 2)) and self.high < 3 << (self.precision - 2):
                self.low -= 1 << (self.precision - 2)
                self.high -= 1 << (self.precision - 2)
                self.value -= 1 << (self.precision - 2)
            else:
                break
            self.low = (self.low << 1) & ((1 << self.precision) - 1)
            self.high = ((self.high << 1) | 1) & ((1 << self.precision) - 1)
            self.value = (self.value << 1) | self._read_bit()
        return symbol


# === Модель PPM с разными способами оценки ухода ↑ ===
class PPMModel:
    def __init__(self, order, alphabet, escape_method='C'):
        self.order = order
        self.alphabet = set(alphabet)
        self.contexts = defaultdict(lambda: defaultdict(int))
        self.escape_method = escape_method

    def update(self, context, symbol):
        self.contexts[tuple(context)][symbol] += 1

    def get_context_freqs(self, context, excluded):
        ctx = tuple(context)
        symbols = set(self.contexts[ctx].keys()) - excluded
        m = len(symbols)
        freqs = {}

        method = self.escape_method.upper()

        if method == 'A':
            for s in symbols:
                freqs[s] = self.contexts[ctx][s]
            if m > 0:
                freqs[ESCAPE_SYMBOL] = 1

        elif method == 'B':
            for s in symbols:
                if self.contexts[ctx][s] > 1:
                    freqs[s] = self.contexts[ctx][s] - 1
            if m > 0:
                freqs[ESCAPE_SYMBOL] = m

        elif method == 'C':
            for s in symbols:
                freqs[s] = self.contexts[ctx][s]
            if m > 0:
                freqs[ESCAPE_SYMBOL] = m

        elif method == 'D':
            # Используем масштабирование для дробей
            scale = 100
            for s in symbols:
                freqs[s] = max(int((self.contexts[ctx][s] - 0.5) * scale), 1)
            if m > 0:
                freqs[ESCAPE_SYMBOL] = int(m / 2 * scale)

        elif method == 'X':
            m1 = sum(1 for cnt in self.contexts[ctx].values() if cnt == 1)
            for s in symbols:
                # freqs[s] = self.contexts[ctx][s]
                freqs[s] = 1
            if m > 0:
                freqs[ESCAPE_SYMBOL] = m1 if m1 > 0 else 1


        else:
            raise ValueError(f"Unknown escape method: {method}")

        return freqs

    def get_order_minus1_freqs(self, excluded):
        symbols = list(self.alphabet - excluded) + [EOF_SYMBOL]
        return {s: 1 for s in symbols}


# === Кодирование текста ===
def ppm_encode(text, order=3, escape_method='C'):
    text = list(text) + [EOF_SYMBOL]
    alphabet = set(text)
    model = PPMModel(order, alphabet, escape_method)
    encoder = ArithmeticEncoder()

    buffer = []

    for symbol in text:
        excluded = set()
        for o in range(order, -2, -1):
            ctx = buffer[-o:] if o > 0 else []
            if o == -1:
                freqs = model.get_order_minus1_freqs(excluded)
            else:
                freqs = model.get_context_freqs(ctx, excluded)
            if not freqs:
                continue
            freq_table = FrequencyTable(list(freqs.keys()))
            freq_table.set_freqs(freqs)
            if symbol in freqs:
                encoder.encode_symbol(symbol, freq_table)
                break
            else:
                encoder.encode_symbol(ESCAPE_SYMBOL, freq_table)
                excluded.update(set(freqs.keys()) - {ESCAPE_SYMBOL})
        # Обновляем модель
        for o in range(order + 1):
            ctx = tuple(buffer[-o:]) if o > 0 else ()
            model.update(ctx, symbol)
        buffer.append(symbol)
        if len(buffer) > order:
            buffer.pop(0)
    encoded_bits = encoder.finish()
    return encoded_bits, alphabet


# === Декодирование битового потока ===
def ppm_decode(bits, alphabet, order=3, escape_method='C'):
    model = PPMModel(order, alphabet, escape_method)
    decoder = ArithmeticDecoder(bits)
    buffer = []
    result = []

    while True:
        excluded = set()
        for o in range(order, -2, -1):
            ctx = buffer[-o:] if o > 0 else []
            if o == -1:
                freqs = model.get_order_minus1_freqs(excluded)
            else:
                freqs = model.get_context_freqs(ctx, excluded)
            if not freqs:
                continue
            freq_table = FrequencyTable(list(freqs.keys()))
            freq_table.set_freqs(freqs)
            symbol = decoder.decode_symbol(freq_table)
            if symbol == ESCAPE_SYMBOL:
                excluded.update(set(freqs.keys()) - {ESCAPE_SYMBOL})
                continue
            if symbol == EOF_SYMBOL:
                return ''.join(result)
            result.append(symbol)
            for o in range(order + 1):
                ctx = tuple(buffer[-o:]) if o > 0 else ()
                model.update(ctx, symbol)
            buffer.append(symbol)
            if len(buffer) > order:
                buffer.pop(0)
            break


# === Пример использования ===
# if __name__ == "__main__":
#     text = "abracadabra"
#     print("Исходный текст:", text)

#     for method in ['A', 'B', 'C', 'D', 'X']:
#         print(f"\nМетод: {method}")
#         bits, alphabet = ppm_encode(text, order=3, escape_method=method)
#         decoded = ppm_decode(bits, alphabet, order=3, escape_method=method)
#         print("Восстановленный текст:", decoded)
#         print("Длина битового потока:", len(bits))
#         assert text == decoded, f"Ошибка декодирования для метода {method}"

In [4]:
with open("Results_text.txt", "w") as file:
    file.write('Результаты работы PPM\n')
m={}
bestsizes={}
i=1
for text in [text1, text2, text3, text4]:
    with open("Results_text.txt", "a") as file:
        file.write(f"\n\n\nИсходный текст: {text[:100]}...\nДлина: {len(text)} символов\nИсходный размер: {main_size} байт")
    m.clear()

    # Сжимаем с разными методами ухода
    for method in ['A', 'B', 'C', 'D', 'X']:
        # Сжатие
        bits, alphabet = ppm_encode(text, order=3, escape_method=method)
        print(f"Сжатые данные: {len(bits)/8} байт ({len(bits)/8/main_size*100:.2f}% от исходного)")
        m[method]= len(bits)/8
        with open("Results_text.txt", "a") as file:
            file.write(f"\nМетод ухода: {method}\nСжатые данные: {len(bits)/8} байт ({(len(bits)/8)/main_size*100:.2f}% от исходного)")

        restored = ppm_decode(bits, alphabet, order=3, escape_method=method)
        with open("Results_text.txt", "a") as file:
            file.write(f"\nДекодировано успешно {text==restored}")

        if text==text2:
            decode_text=restored
    
    with open("Results_text.txt", "a") as file:
        min_value = min(m.values())
        min_keys = [k for k, v in m.items() if v == min_value]
        bestsizes[i]={min(m, key=m.get):min_value}
        # bestsizes.append(min_value)
        if len(min_keys)==1:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показал метод оценки вероятности ухода {min(m, key=m.get)} - {min_value} байт")
        else:
            file.write(f"\n\nВывод по методу сжатия: Наилучший результат показали методы оценки вероятности ухода {', '.join(map(str, min_keys))} - {min_value} байт")
    
    i+=1

        

Сжатые данные: 912106.25 байт (28.76% от исходного)
Сжатые данные: 919785.875 байт (29.00% от исходного)
Сжатые данные: 914029.125 байт (28.82% от исходного)
Сжатые данные: 911102.875 байт (28.73% от исходного)
Сжатые данные: 1506443.5 байт (47.50% от исходного)
Сжатые данные: 906401.375 байт (28.58% от исходного)
Сжатые данные: 913772.375 байт (28.81% от исходного)
Сжатые данные: 909854.25 байт (28.69% от исходного)
Сжатые данные: 906885.0 байт (28.59% от исходного)
Сжатые данные: 1488327.5 байт (46.93% от исходного)
Сжатые данные: 912636.0 байт (28.78% от исходного)
Сжатые данные: 920750.75 байт (29.03% от исходного)
Сжатые данные: 914296.375 байт (28.83% от исходного)
Сжатые данные: 911706.5 байт (28.75% от исходного)
Сжатые данные: 1503839.125 байт (47.42% от исходного)
Сжатые данные: 905525.0 байт (28.55% от исходного)
Сжатые данные: 913194.0 байт (28.79% от исходного)
Сжатые данные: 909292.25 байт (28.67% от исходного)
Сжатые данные: 906420.875 байт (28.58% от исходного)
Сжатые д

Дополнительная кодировка массива исключения для декапитализированного текста

In [5]:
def generate_bit_array(text, extra): # Применение общих правил декапитализации и создание битового массива, в котором 1 указывают на позиции исключений
    bit_array = []
    text_list = list(text)  # Преобразуем строку в список для изменения символов
    # print(text)
    expected_lowers=[True]*len(text)
    for i, char in enumerate(text):
        if char.isalpha():
            
            expected_lower = True  # по умолчанию ожидаем строчную
            
            if extra:
                # Проверка на название (2 заглавние подряд)
                if (i>=2 and ((text[i - 2].isalpha() and text[i-2].isupper()) and (text[i-1].isalpha() and text[i-1].isupper()))) and text_list[i].isupper():
                    expected_lower = False
                    text_list[i]=text_list[i].lower()

            # Проверка правил
            if (i == 0 or (i > 1 and text[i - 2:i] in {". ", "! ", "? ", "\n"}) or (i > 1 and text[i - 1:i] in {'"'})) and text_list[i].isupper():
                expected_lower = False
                text_list[i]=text_list[i].lower()
                # caps.append(i)

            # Проверка местоимения 'I'
            if (char == 'I' and (i == 0 or not text[i - 1].isalpha()) and (
                    i == len(text) - 1 or not text[i + 1].isalpha())) and text_list[i].isupper():
                expected_lower = False
                text_list[i]=text_list[i].lower()
                # caps.append(i)

            # Проверка исключений
            if (char.islower() and not expected_lower) or (char.isupper() and expected_lower):
                bit_array.append(1)
                # caps.append(i)
            else:
                bit_array.append(0)

            expected_lowers[i]=expected_lower
        else:
            bit_array.append(0)  # не буквы игнорируем

    text_new = ''.join(text_list)
    return bit_array, text_new

In [6]:
rules, text_decap=generate_bit_array(text1, True)
print('Исходный размер массива исключений:')
# print(rules[:100])
size_of_exceptions=len(rules)/8 # в байтах
# print(len(rules))
bytes_info(size_of_exceptions)

# закодируем количество нулей между 1
numbers=[]
z=0
i=0
while i+1<=len(rules):
    if rules[i]==0:
        z+=1
    else:
        # print(z)
        numbers.append(z)
        z=0
    i+=1
    if i+1==len(rules):
        numbers.append(z)
        break

Исходный размер массива исключений:
Размер файла: 396441.125 байт или  387.15 Кб или  0.378 Мб


Для кодирования массива исключений используем дельта-кодирование Элиаса, так как оно показало хороший результат на рассматриваемых данных (задание 2)

In [7]:
# Дельта-кодирование Элиаса (кодер и декодер)
def elias_delta_encode(n):
    if n == 0:
        return '0'
    # if n < 1:
    #     raise ValueError("Дельта-код Элиаса работает только с натуральными числами (n >= 1)")
    
    # 1. Кодируем длину числа (L) с помощью гамма-кода
    if n == 1:
        return '1'
    
    binary = bin(n)[2:]  # Двоичное представление числа без '0b'
    L = len(binary)
    
    # 2. Кодируем длину длины (LL = floor(log2(L)) + 1)
    LL_bin = bin(L)[2:]
    LL = len(LL_bin)
    
    # 3. Формируем код:
    # - (LL-1) нулей
    # - затем LL-битное представление L
    # - затем (L-1) младших битов числа n (без старшей 1)
    return '0'*(LL-1) + LL_bin + binary[1:]

def elias_delta_decode(encoded):
    if encoded == '0':
        return 0
    if encoded == '1':
        return 1
    
    # 1. Читаем длину длины (LL)
    zero_count = 0
    while encoded[zero_count] == '0':
        zero_count += 1
    LL = zero_count + 1
    
    # 2. Читаем длину числа (L)
    if zero_count + LL > len(encoded):
        raise ValueError("Некорректный дельта-код")
    
    L_bin = encoded[zero_count:zero_count + LL]
    L = int(L_bin, 2)
    
    # 3. Читаем само число
    number_part = encoded[zero_count + LL:]
    if len(number_part) < L - 1:
        raise ValueError("Некорректный дельта-код")
    
    binary = '1' + number_part[:L-1]
    return int(binary, 2)


In [8]:
print("\nДельта-кодирование Элиаса:")
encoded_sequence = [elias_delta_encode(n) for n in numbers]   
print(encoded_sequence)
s=len(''.join(encoded_sequence))/8 #получившийся размер массива исключений
bytes_info(s)
compare_size(size_of_exceptions, s)


Дельта-кодирование Элиаса:
['1', '01111', '0', '01110', '0', '0101', '0', '01110', '1', '0', '0101', '0', '0100', '0', '00100010', '0101', '01101', '00100000', '00100010', '00100000', '00100000', '00100010', '00100001', '01101', '00111010000', '0001010000010111', '00100101', '000100101000011', '01111', '00010001010111', '00010000011000', '001010011', '0011000010', '00010000010011', '001011010', '0001010010011110', '001011111', '001010001', '01111', '01110', '00010001010011', '00111011011', '01101', '00100000', '001011000', '0011001111', '00111000001', '0101', '00010000111011', '00111011110', '001010110', '001010001', '001011000', '001011110', '00111000100', '00100001', '01101', '00111011110', '00111001101', '001011000', '000100110010001', '00111100000', '00010001111010', '0011001001', '00100100', '00111110101', '00100100', '001010101', '01110', '0011001111', '01101', '0011001101', '001011111', '001011001', '0001010100110010', '00010001000001', '00111001111', '000100100001001', '01110'

In [9]:
with open("Results_text_decap.txt", "w") as file:
    file.write('Результаты работы PPM\n')

    file.write(f"\n\nВывод по методу сжатия (text2): Наилучший результат показал метод оценки вероятности ухода {list(bestsizes[2].keys())[0]} - {list(bestsizes[2].values())[0]+s} байт")
    file.write(f"\n\nСжатые данные: {list(bestsizes[2].values())[0]+s} байт ({(list(bestsizes[2].values())[0]+s)/main_size*100:.2f}% от исходного)")

    file.write(f"\n\nВывод по методу сжатия (text4): Наилучший результат показал метод оценки вероятности ухода {list(bestsizes[4].keys())[0]} - {list(bestsizes[4].values())[0]+s} байт")       
    file.write(f"\n\nСжатые данные: {list(bestsizes[4].values())[0]+s} байт ({(list(bestsizes[4].values())[0]+s)/main_size*100:.2f}% от исходного)")

Восстановление исходного текста

In [10]:
# на примере текста 2 (декапитализированного исходного)
decode_text=text2
print(decode_text[:1000])

volume i fantine book first - a just man chapter i - m. myriel in 1815, m. charles-francois-bienvenu myriel was bishop of d - - he was an old man of about seventy-five years of age; he had occupied the see of d - - since 1806. although this detail has no connection whatever with the real substance of what we are about to relate, it will not be superfluous, if merely for the sake of exactness in all points, to mention here the various rumors and remarks which had been in circulation about him from the very moment when he arrived in the diocese. true or false, that which is said of men often occupies as important a place in their lives, and above all in their destinies, as that which they do. m. myriel was the son of a councillor of the parliament of aix; hence he belonged to the nobility of the bar. it was said that his father, destining him to be the heir of his own post, had married him at a very early age, eighteen or twenty, in accordance with a custom which is rather widely prevale

In [11]:
# декодируем массив расстояний между 1 (исключениями)
decoded_sequence = [elias_delta_decode(code) for code in encoded_sequence]
print('Правильно ли декодировался массив расстояний', decoded_sequence==numbers)

# восстанавливаем битовый массив исключений
exp=[]
l=0
while True:
    for j in range(decoded_sequence[l]):
        exp.append(0)
    if (l+1)!=len(decoded_sequence):
        exp.append(1)
    else: 
        exp.append(0)
        break
    l+=1

print('Правильно ли восстановлен массив исключений', rules==exp)
print(exp[:100])

Правильно ли декодировался массив расстояний True
Правильно ли восстановлен массив исключений True
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [12]:
# Применяем правила для восстановления заглавных букв
def rules_to_text(text, exceptions): # Применение общих правил декапитализации и создание битового массива, в котором 1 указывают на позиции исключений
    text_list = list(text)  # Преобразуем строку в список для изменения символов
    for i, char in enumerate(text):
        expected_lower = True
        if char.isalpha():
            # Проверка правил
            if i == 0 or (i > 1 and text[i - 2:i] in {". ", "! ", "? ", "\n"}) or (i > 1 and text[i - 1:i] in {'"'}):
                expected_lower=False
                text_list[i]=text_list[i].upper()
            # Проверка местоимения 'I'
            if char == 'i' and (i == 0 or not text[i - 1].isalpha()) and (
                    i == len(text) - 1 or not text[i + 1].isalpha()):
                expected_lower=False
                text_list[i]=text_list[i].upper()
            # Проверка на название (2 заглавние подряд)
            if i>=2 and ((text_list[i - 2].isalpha() and text_list[i-2].isupper()) and (text_list[i-1].isalpha() and text_list[i-1].isupper())):
                expected_lower=False
                text_list[i]=text_list[i].upper()
            # поверка исключений
            if (exceptions[i]==1 and expected_lower==True):
                text_list[i]=text_list[i].upper()
            elif (exceptions[i]==1 and expected_lower==False):
                text_list[i]=text_list[i].lower()

    text_new = ''.join(text_list)
    return text_new

In [13]:
# Применяем правила для восстановления заглавных букв
text_restored=rules_to_text(decode_text, exp)
print(text_restored[:1000])

VOLUME I FANTINE BOOK FIRST - A JUST MAN CHAPTER I - M. MYRIEL In 1815, M. Charles-Francois-Bienvenu Myriel was Bishop of D - - He was an old man of about seventy-five years of age; he had occupied the see of D - - since 1806. Although this detail has no connection whatever with the real substance of what we are about to relate, it will not be superfluous, if merely for the sake of exactness in all points, to mention here the various rumors and remarks which had been in circulation about him from the very moment when he arrived in the diocese. True or false, that which is said of men often occupies as important a place in their lives, and above all in their destinies, as that which they do. M. Myriel was the son of a councillor of the Parliament of Aix; hence he belonged to the nobility of the bar. It was said that his father, destining him to be the heir of his own post, had married him at a very early age, eighteen or twenty, in accordance with a custom which is rather widely prevale